In [1]:
import pandas as pd

In [2]:
ahmedabad_driver_data = pd.read_csv("Events Report of Drivers in Ahmedabad_16-07 to 31-07.csv")

In [ ]:
ahmedabad_driver_data.head()

In [ ]:
ahmedabad_driver_data.info()

## Check if the current point lies in the danzer zone

In [3]:
from math import radians, cos, sin, asin, sqrt

def haversine(long1, lat1, long2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    long1, lat1, long2, lat2 = map(radians, [long1, lat1, long2, lat2])

    # haversine formula 
    dlong = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlong/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers.
    return c * r

## Harsh Braking Radius
##### 30mt

In [4]:
ahmedabad_driver_data_harsh = ahmedabad_driver_data[ahmedabad_driver_data["Event Name"] == "Harsh Braking"]
ahmedabad_driver_data_harsh.reset_index(drop=True, inplace=True)

In [ ]:
ahmedabad_driver_data_harsh.groupby("Route Name").count()

#### Sample route

In [5]:
ahmedabad_driver_data_harsh_smaple = ahmedabad_driver_data_harsh[ahmedabad_driver_data_harsh["Route Name"] == 
                                                            "11D_Odhav Ring Road - M J Library_NORMAL"]
ahmedabad_driver_data_harsh_smaple.reset_index(drop=True, inplace=True)

In [6]:
ahmedabad_driver_data_harsh_smaple.shape

(793, 10)

In [ ]:
ahmedabad_driver_data_harsh_smaple = ahmedabad_driver_data_harsh_smaple.iloc[:100, :]

In [ ]:
t_harsh_braking_df_test = ahmedabad_driver_data_harsh_smaple[["Bus no.", "Event Time", "Latitude", "Longitude"]]
t_harsh_braking_df_test = t_harsh_braking_df_test.reindex(columns=t_harsh_braking_df_test.columns.tolist() + 
                                                ["count", "count_unique_bus", "count_current_bus"])

In [7]:
def cal(dataset):
    global t_harsh_braking, t_harsh_braking_unique_bus, t_harsh_braking_current_bus,\
           count, count_unique_bus, count_current_bus, center_point
        
    center = {"Latitude": dataset[8], "Longitude": dataset[9], "Bus no.": dataset[4], "Event Time": dataset[1]}
    center_point = {'lat': center["Latitude"], 'long': center["Longitude"]}
    
    bus_no = center["Bus no."]
    event_time = center["Event Time"]
    coordinates = ', '.join(map(str, center_point.values()))
    count = 0
    count_unique_bus = 0
    count_current_bus = 0
    bus_nos = [center["Bus no."]]
    
    def calc(dataset):
        global count, count_unique_bus, count_current_bus, center_point
        test = {"Latitude": dataset[8], "Longitude": dataset[9], "Bus no.": dataset[4]}
        test_point = {'lat': test["Latitude"], 'long': test["Longitude"]}

        lat1 = center_point['lat']
        long1 = center_point['long']
        lat2 = test_point['lat']
        long2 = test_point['long']

        radius = 0.030 # in kilometer
        a = haversine(long1, lat1, long2, lat2)

        if a <= radius:
            count += 1
            if test["Bus no."] not in bus_nos:
                bus_nos.append(dataset[4])
                count_unique_bus += 1
            if test["Bus no."] == center["Bus no."]:
                count_current_bus += 1
    
    ahmedabad_driver_data_harsh_smaple.apply(calc, axis=1)
    
    
#     t_harsh_braking["bus_no"].append(bus_no)
#     t_harsh_braking["event_time"].append(event_time)    
#     t_harsh_braking["coordinates"].append(coordinates)    
#     t_harsh_braking["count"].append(count)  
    
#     t_harsh_braking_unique_bus["bus_no"].append(bus_no)
#     t_harsh_braking_unique_bus["event_time"].append(event_time)    
#     t_harsh_braking_unique_bus["coordinates"].append(coordinates)    
#     t_harsh_braking_unique_bus["count"].append(count_unique_bus)  
    
#     t_harsh_braking_current_bus["bus_no"].append(bus_no)
#     t_harsh_braking_current_bus["event_time"].append(event_time)    
#     t_harsh_braking_current_bus["coordinates"].append(coordinates)    
#     t_harsh_braking_current_bus["count"].append(count_current_bus)    
    
    return [count, count_unique_bus, count_current_bus]

In [8]:
t_harsh_braking_df_test[["count", "count_unique_bus", "count_current_bus"]] = ahmedabad_driver_data_harsh_smaple.\
                                                                              apply(cal, axis=1).tolist()

In [ ]:
# t_harsh_braking = {"bus_no": [], "event_time": [], "coordinates": [], "count": []}
# t_harsh_braking_unique_bus = {"bus_no": [], "event_time": [], "coordinates": [], "count": []}
# t_harsh_braking_current_bus = {"bus_no": [], "event_time": [], "coordinates": [], "count": []}

# # ahmedabad_driver_data_harsh_smaple.apply(cal, axis=1)
# print("done")

In [ ]:
# t_harsh_braking_df = pd.DataFrame(t_harsh_braking, 
#                                   columns=["bus_no", "event_time", "coordinates", "count"])
# t_harsh_braking_unique_bus_df = pd.DataFrame(t_harsh_braking_unique_bus, 
#                                   columns=["bus_no", "event_time", "coordinates", "count"])
# t_harsh_braking_current_bus_df = pd.DataFrame(t_harsh_braking_current_bus, 
#                                   columns=["bus_no", "event_time", "coordinates", "count"])

In [ ]:
t_harsh_braking_df_test.to_csv("Total_harsh_braking.csv", index=False)

In [ ]:
t_harsh_braking_df.to_csv("Total_harsh_braking.csv", index=False)
t_harsh_braking_unique_bus_df.to_csv("Total_harsh_braking_with_unique_bus.csv", index=False)
t_harsh_braking_current_bus_df.to_csv("Total_harsh_braking_current_bus.csv", index=False)